In [1]:
import torch
import torchani

In [2]:
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

In [3]:
Rcr = 5.2000e+00
Rca = 3.5000e+00
EtaR = torch.tensor([1.6000000e+01], device=device)
ShfR = torch.tensor([9.0000000e-01, 1.1687500e+00, 1.4375000e+00, 1.7062500e+00, 1.9750000e+00, 2.2437500e+00, 2.5125000e+00, 2.7812500e+00, 3.0500000e+00, 3.3187500e+00, 3.5875000e+00, 3.8562500e+00, 4.1250000e+00, 4.3937500e+00, 4.6625000e+00, 4.9312500e+00], device=device)
Zeta = torch.tensor([3.2000000e+01], device=device)
ShfZ = torch.tensor([1.9634954e-01, 5.8904862e-01, 9.8174770e-01, 1.3744468e+00, 1.7671459e+00, 2.1598449e+00, 2.5525440e+00, 2.9452431e+00], device=device)
EtaA = torch.tensor([8.0000000e+00], device=device)
ShfA = torch.tensor([9.0000000e-01, 1.5500000e+00, 2.2000000e+00, 2.8500000e+00], device=device)
species_order = ['H', 'C', 'N', 'O']
num_species = len(species_order)
aev_computer = torchani.AEVComputer(Rcr, Rca, EtaR, ShfR, EtaA, Zeta, ShfA, ShfZ, num_species)
energy_shifter = torchani.utils.EnergyShifter(None)

In [4]:
aev_dim = aev_computer.aev_length

H_network = torch.nn.Sequential(
    torch.nn.Linear(aev_dim, 160),
    torch.nn.CELU(0.1),
    torch.nn.Linear(160, 128),
    torch.nn.CELU(0.1),
    torch.nn.Linear(128, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

# C_network = torch.nn.Sequential(
#     torch.nn.Linear(aev_dim, 144),
#     torch.nn.CELU(0.1),
#     torch.nn.Linear(144, 112),
#     torch.nn.CELU(0.1),
#     torch.nn.Linear(112, 96),
#     torch.nn.CELU(0.1),
#     torch.nn.Linear(96, 1)
# )
C_network = torch.nn.Sequential(
    torch.nn.Linear(aev_dim, 112),
    torch.nn.CELU(0.1),
    torch.nn.Linear(112, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

N_network = torch.nn.Sequential(
    torch.nn.Linear(aev_dim, 128),
    torch.nn.CELU(0.1),
    torch.nn.Linear(128, 112),
    torch.nn.CELU(0.1),
    torch.nn.Linear(112, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

# O_network = torch.nn.Sequential(
#     torch.nn.Linear(aev_dim, 128),
#     torch.nn.CELU(0.1),
#     torch.nn.Linear(128, 112),
#     torch.nn.CELU(0.1),
#     torch.nn.Linear(112, 96),
#     torch.nn.CELU(0.1),
#     torch.nn.Linear(96, 1)
# )
O_network = torch.nn.Sequential(
    torch.nn.Linear(aev_dim, 112),
    torch.nn.CELU(0.1),
    torch.nn.Linear(112, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

#nn = torchani.ANIModel([H_network, C_network, N_network, O_network])
nn = torchani.ANIModel([C_network, O_network])

In [5]:
model = torch.load('save_model_1.pt')
model.eval()

Sequential(
  (0): AEVComputer()
  (1): ANIModel(
    (0): Sequential(
      (0): Linear(in_features=384, out_features=160, bias=True)
      (1): CELU(alpha=0.1)
      (2): Linear(in_features=160, out_features=128, bias=True)
      (3): CELU(alpha=0.1)
      (4): Linear(in_features=128, out_features=96, bias=True)
      (5): CELU(alpha=0.1)
      (6): Linear(in_features=96, out_features=1, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=384, out_features=144, bias=True)
      (1): CELU(alpha=0.1)
      (2): Linear(in_features=144, out_features=112, bias=True)
      (3): CELU(alpha=0.1)
      (4): Linear(in_features=112, out_features=96, bias=True)
      (5): CELU(alpha=0.1)
      (6): Linear(in_features=96, out_features=1, bias=True)
    )
    (2): Sequential(
      (0): Linear(in_features=384, out_features=128, bias=True)
      (1): CELU(alpha=0.1)
      (2): Linear(in_features=128, out_features=112, bias=True)
      (3): CELU(alpha=0.1)
      (4): Linear(in_featur

In [6]:
D = [i/100 for i in range(1,250)]
energyData = []

for i in D:
    coordinates = torch.tensor([[[0.0, 0.0, 0.0],
                             [0.0, 0.0, i ]]],
                           requires_grad=True, device=device)
    # In periodic table, C = 6 and O = 8
    species = torch.tensor([[6, 8]], device=device)

    energy = model((species, coordinates)).energies
    energyData.append(energy.item())

IndexError: index out of range in self